In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.preprocessing import RobustScaler
import statistics
import math

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%ls

drive/  sample_data/


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/7313Project/test_reduced.csv")
train = pd.read_csv("/content/drive/MyDrive/7313Project/train_reduced.csv")
testn = pd.read_csv("/content/drive/MyDrive/7313Project/test_reduced.csv")
target = pd.read_csv("/content/drive/MyDrive/7313Project/target.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
train.drop(train.iloc[:, [0,1]], axis=1, inplace=True)
test.drop(test.iloc[:, [0,1]], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [ ]:
train['is_cup'] = labelencoder.fit_transform(train['is_cup'])
test['is_cup'] = labelencoder.fit_transform(test['is_cup'])
target['target'].replace({"home": 0, "draw": 1, "away":2}, inplace=True)
train.drop(train.filter(regex='recorded_matches').columns, axis=1, inplace = True)
test.drop(test.filter(regex='recorded_matches').columns, axis=1, inplace = True)
train_x = train.copy()
train_y = target['target'].copy()
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

#fit scaler on training data
norm = StandardScaler().fit(train_x)
#transform training data
xtrain_norm = norm.transform(train_x)
#transform testing data
test_norm = norm.transform(test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

# X_train, X_test, y_train, y_test = train_test_split(
#     xtrain_norm, train_y, test_size=0.3, random_state=2, stratify=train_y)
tree = DecisionTreeClassifier(criterion='entropy', max_depth=1)
bc = AdaBoostClassifier(base_estimator=tree,
                           n_estimators=500,
                           algorithm='SAMME',
                           random_state=1)
bc.fit(xtrain_norm, train_y)

y_pred = bc.predict(xtrain_norm)
acc = np.mean(y_pred == train_y)*100
acc

49.54479078404154

In [ ]:
from mlxtend.evaluate import confusion_matrix
confmat = confusion_matrix(train_y, y_pred)

print(confmat)

[[38046   307  9760]
 [18577   351  8723]
 [18333   274 16567]]


In [ ]:
prediction_1 = bc.predict_proba(test_norm)
prediction_1

array([[0.33711172, 0.33182997, 0.33105831],
       [0.3358022 , 0.33260819, 0.33158961],
       [0.33856311, 0.33108596, 0.33035093],
       ...,
       [0.3277463 , 0.32895253, 0.34330117],
       [0.35173505, 0.33031683, 0.31794812],
       [0.3327427 , 0.33201623, 0.33524107]])

In [ ]:
submission = pd.concat([pd.DataFrame(testn['id']), pd.DataFrame(prediction_1, columns=['home', 'draw', 'away'])[['home', 'draw', 'away']]], axis=1)
submission.to_csv('/content/drive/MyDrive/7313Project/submission_adaboost.csv', index=False)